In [1]:
# coding: utf-8
import pandas as pd
import numpy as np
import nltk
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.metrics.distance import *
from gensim.models import Word2Vec, KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet as wn

sentence = 'this is a foo bar sentences and i want to ngramize it'
n = 6
sixgrams = ngrams(sentence.split(), n)
for grams in sixgrams:
    print grams

('this', 'is', 'a', 'foo', 'bar', 'sentences')
('is', 'a', 'foo', 'bar', 'sentences', 'and')
('a', 'foo', 'bar', 'sentences', 'and', 'i')
('foo', 'bar', 'sentences', 'and', 'i', 'want')
('bar', 'sentences', 'and', 'i', 'want', 'to')
('sentences', 'and', 'i', 'want', 'to', 'ngramize')
('and', 'i', 'want', 'to', 'ngramize', 'it')


In [2]:
def get_ngram_word_overlap(s1,s2,ngram):


    ngrams_list_s1 = set(ngrams(s1, ngram))
    ngrams_list_s2 = set(ngrams(s2, ngram))
    
    s1_s2_intersection = list(ngrams_list_s1 & ngrams_list_s2)
    
#     print(s1_s2_intersection)
    
    word_overlap = (2 * (np.divide(len(ngrams_list_s1), len(s1_s2_intersection)) +
                        np.divide(len(ngrams_list_s2), len(s1_s2_intersection)))) - 1
    
    return word_overlap

def get_ngram_char_overlap(s1,s2,ngram):
    
    ngrams_list_s1 = set(["".join(j) for j in zip(*[s1[i:] for i in range(ngram)])])
    ngrams_list_s2 = set(["".join(j) for j in zip(*[s2[i:] for i in range(ngram)])])
    
    s1_s2_intersection = list(ngrams_list_s1 & ngrams_list_s2)
    
#     print(s1_s2_intersection)
    
    char_overlap = (2 * (np.divide(len(ngrams_list_s1), len(s1_s2_intersection)) +
                        np.divide(len(ngrams_list_s2), len(s1_s2_intersection)))) - 1
    return char_overlap



s1 = 'The villains I absolutely hate are selfish and self-serving, but they are also cowards.'
s2 = 'A villain you want to take down is, at his/her core, someone who does not care about the suffering of others.'
n = 1

get_ngram_word_overlap(s1, s2, n)
    
    

3

In [3]:
def longestCommonPrefix(strs):
        
        if not strs:
            return ""
        for word_list in strs:
            ' '.join(word for word in word_list)
        for i in xrange(len(strs[0])):
            for string in strs[1:]:
                if i >= len(string) or string[i] != strs[0][i]:
                    return len(strs[0][:i])
        return len(strs[0])
    
def longestCommonSuffix(strs):
        
        if not strs:
            return ""
        for word_list in strs:
            ' '.join(word for word in word_list)
        s1_len = len(strs[0])
        s2_len = len(strs[1])
        while s1_len > 0 and s2_len > 0:
            if strs[1][s2_len - 1] != strs[0][s1_len - 1]:
                return len(strs[0][s1_len - 1:-1])
            s1_len -= 1; 
            s2_len -= 1;
        return len(strs[0])
    
def longestSubstringFinder(s1, s2):
    answer = ""
    len1, len2 = len(s1), len(s2)
    s1 = ' '.join(word for word in s1)
    s2 = ' '.join(word for word in s2)
    for i in range(len1):
        match = ""
        for j in range(len2):
            if (i + j < len1 and s1[i + j] == s2[j]):
                match += s2[j]
            else:
                if (len(match) > len(answer)): answer = match
                match = ""
           
    return len(answer)

def get_levenshtein_distance(s1, s2):
    return edit_distance(s1,s2)


def get_binary_distance(s1, s2):
    return binary_distance(s1,s2)

def get_jaccard_distance(s1, s2):
    return jaccard_distance(s1,s2)

def get_masi_distance(s1, s2):
    return masi_distance(s1,s2)

def get_interval_distance(s1, s2):
    return interval_distance(s1,s2)




s1 = word_tokenize('The bird is bathing in the sink.')
s2 = word_tokenize('Birdie is washing itself in the water basin.')
print s1
print s2
print get_jaccard_distance(set(s1),set(s2))

# get_masi_distance(s1,s2)
# # get_jaccard_distance('The bird is bathing in the sink.','Birdie is washing itself in the water basin.')

['The', 'bird', 'is', 'bathing', 'in', 'the', 'sink', '.']
['Birdie', 'is', 'washing', 'itself', 'in', 'the', 'water', 'basin', '.']
0.692307692308


In [4]:
model = KeyedVectors.load_word2vec_format('../../../../GoogleNews-vectors-negative300.bin.gz', binary=True, limit=600000)

def convert2BOWV(word_list):
    wv_list = []
    
    for w in word_list:
        if w in model.vocab:
            wv_list.append(model[w]) 
    return wv_list

# sum of all word vectors
def getSentenceEmbedings(word_list):
    
    wv_list = convert2BOWV(word_list)
    if wv_list:
        sv = wv_list[0]
        for i in range(len(wv_list)):
            if i != 0:
                sv = np.add(sv,wv_list[i])
        return sv.reshape(1, -1)  
    else:
        return np.zeros((1,300))
    


In [59]:
error_count = 0
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('V'):
        return wn.VERB
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('I'):
        return 'p'
    elif tag.startswith('CD'):
        return 'cd'
    else:
        return None
def change_tag_to_WNpos(sentence):
    tagged_sent = []
    for w, tag in sentence:
        tagged_sent.append([w,get_wordnet_pos(tag)])
    return tagged_sent    
    
def get_alignment_score(s1,s2, alignment, pos=None):
    
    if alignment == None:
        print s1, s2
        return
    if pos is None:
        s1_cnt = len(set(s1))
        s2_cnt = len(set(s2))
        sa1_cnt = 0
        sa2_cnt = 0
        for a in alignment:
            try:
                sa1_cnt += 1
                sa2_cnt += 1
            except:
                print 'errror' 
                
        align_score = np.divide(float(sa1_cnt + sa2_cnt),float(s1_cnt + s2_cnt))
        return align_score
    else:
        
        s1_pos = change_tag_to_WNpos(nltk.pos_tag(s1))
        s2_pos = change_tag_to_WNpos(nltk.pos_tag(s2))
        
        pos_tag_s1 = sum([1 for c,s in enumerate(s1) if s1_pos[c][1] == pos])
        pos_tag_s2 = sum([1 for c,s in enumerate(s2) if s2_pos[c][1] == pos])
                
        sa1_cnt = 0
        sa2_cnt = 0 
        
        alignment_map = {}
        for c,a in enumerate(alignment):
            alignment_map[a[0]] = a[1]
        for idx, sent1_pos in enumerate(s1_pos):
            try:
                if idx in alignment_map and sent1_pos[1] == pos:
                    sa1_cnt += 1
                    sa2_cnt += 1 
            except:
                   print 'errror'
        align_score = np.divide(float(sa1_cnt + sa2_cnt),float(pos_tag_s1 + pos_tag_s2))
        
        
        return align_score
    
    
    
alignment = [[7, 8], [2, 2], [3, 4], [1, 1], [5, 6]]
test_align = [['.', '.'], ['men', 'people'], ['died', 'dead'], ['Four', '4'], ['accident', 'collision'], ['an', 'a']]
s1 = word_tokenize("Four men died in an accident.")
s2 = word_tokenize("4 people are dead from a collision.")

# get_alignment_score(s1,s2,alignment,)
print s1
print s2
print change_tag_to_WNpos(nltk.pos_tag(s1))
print change_tag_to_WNpos(nltk.pos_tag(s2))
print nltk.pos_tag(s1)
print nltk.pos_tag(s2)
print get_alignment_score(s1,s2,alignment, 'n')

['Four', 'men', 'died', 'in', 'an', 'accident', '.']
['4', 'people', 'are', 'dead', 'from', 'a', 'collision', '.']
[['Four', 'cd'], ['men', u'n'], ['died', u'v'], ['in', 'p'], ['an', None], ['accident', u'n'], ['.', None]]
[['4', 'cd'], ['people', u'n'], ['are', u'v'], ['dead', u'a'], ['from', 'p'], ['a', None], ['collision', u'n'], ['.', None]]
[('Four', 'CD'), ('men', 'NNS'), ('died', 'VBD'), ('in', 'IN'), ('an', 'DT'), ('accident', 'NN'), ('.', '.')]
[('4', 'CD'), ('people', 'NNS'), ('are', 'VBP'), ('dead', 'JJ'), ('from', 'IN'), ('a', 'DT'), ('collision', 'NN'), ('.', '.')]
1.0


In [6]:
# from nltk.parse.stanford import GenericStanfordParser

# g = GenericStanfordParser()
# print g.tagged_parse(nltk.pos_tag('Four men died in an accident.'))
# from nltk.tokenize.moses import MosesDetokenizer,MosesTokenizer
# from nltk.tokenize import TweetTokenizer
# from nltk.tokenize import PunktWordTokenizer
# ptoken = PunktWordTokenizer()
# tknzr =MosesTokenizer()
# detokenizer = MosesDetokenizer()
# def sentence_tokenise(s):
#     s = detokenizer.detokenize(s, return_str=True)
#     s =  ptoken.tokenize(s)
#     s = detokenizer.detokenize(s, return_str=True)
#     return tknzr.tokenize(s)        

In [151]:
# import sys
# reload(sys)
# sys.setdefaultencoding('utf8')
# def encode(s):
#     return s.encode('utf-8')
# df = pd.read_pickle("Preprocessed_ML_Align_TEST.pickle")
# df.to_excel("Preprocessed_ML_Align_TEST.xlsx")
# print "start"

df = pd.read_pickle("Preprocessed_ML_Align_TRAIN.pickle")
df.to_excel("Preprocessed_ML_Align_TRAIN.xlsx")
print "start"

# df['Sentence1'] = df.apply(lambda x: sentence_tokenise(x['Sentence1']), axis=1)
# df['Sentence2'] = df.apply(lambda x: sentence_tokenise(x['Sentence2']), axis=1)
# df['Sentence1'] = df.apply(lambda x: encode(x['Sentence1']), axis=1)
# df['Sentence2'] = df.apply(lambda x: encode(x['Sentence2']), axis=1)

start


In [152]:
df["1gram_woverlap"] = df.apply(lambda x: get_ngram_word_overlap(x['Sentence1'], x['Sentence2'],1), axis=1)
df["2gram_woverlap"] = df.apply(lambda x: get_ngram_word_overlap(x['Sentence1'], x['Sentence2'],2), axis=1)
df["3gram_woverlap"] = df.apply(lambda x: get_ngram_word_overlap(x['Sentence1'], x['Sentence2'],3), axis=1)

df["2gram_coverlap"] = df.apply(lambda x: get_ngram_char_overlap(x['Sentence1'], x['Sentence2'],2), axis=1)
df["3gram_coverlap"] = df.apply(lambda x: get_ngram_char_overlap(x['Sentence1'], x['Sentence2'],3), axis=1)
df["4gram_coverlap"] = df.apply(lambda x: get_ngram_char_overlap(x['Sentence1'], x['Sentence2'],4), axis=1)
df["5gram_coverlap"] = df.apply(lambda x: get_ngram_char_overlap(x['Sentence1'], x['Sentence2'],5), axis=1)

/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in divide
  # This is added back by InteractiveShellApp.init_path()
/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in divide
  if sys.path[0] == '':
/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in divide
/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in divide


In [153]:
df["longestCommonPrefix"] = df.apply(lambda x: longestCommonPrefix([x['Sentence1'], x['Sentence2']]), axis=1)
df["longestCommonSuffix"] = df.apply(lambda x: longestCommonSuffix([x['Sentence1'], x['Sentence2']]), axis=1)
df["longestSubstringFinder"] = df.apply(lambda x: longestSubstringFinder(x['Sentence1'], x['Sentence2']), axis=1)

In [154]:
df["levenshtein_distance"] = df.apply(lambda x: get_levenshtein_distance(x['Sentence1'], x['Sentence2']), axis=1)
# df["binary_distance"] = df.apply(lambda x: get_binary_distance(x['Sentence1'], x['Sentence2']), axis=1)

In [155]:
df["jaccard_distance"] = df.apply(lambda x: get_jaccard_distance(set(x['Sentence1']), set(x['Sentence2'])), axis=1)
df["masi_distance"] = df.apply(lambda x: masi_distance(set(x['Sentence1']), set(x['Sentence2'])), axis=1)
# df["interval_distance"] = df.apply(lambda x: get_interval_distance(set(x['Sentence1']), set(x['Sentence2'])), axis=1)

In [156]:

df['cos_sim'] = df.apply(lambda x: 
                         cosine_similarity(getSentenceEmbedings(x['Sentence1']), 
                                           getSentenceEmbedings(x['Sentence2'])).item(0), axis=1)

In [157]:
def get_align(x):
    
    try:
        return x['Alignment'][0]
    except:
        return []
a = {}        
a['Alignment'] = [[[10, 4], [4, 3], [2, 2]], [['.', '.'], ['voice', 'sound'], ['high-pitched', 'high-pitched']]]        
print get_align(a);        

[[10, 4], [4, 3], [2, 2]]


In [158]:
df['Alignment'] = df.apply(lambda x: get_align(x), axis=1)

In [159]:

df['align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment']), axis=1)
df['N_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'n'), axis=1)
df['V_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'v'), axis=1)
df['R_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'r'), axis=1)
df['A_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'a'), axis=1)
df['P_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'p'), axis=1)
df['CD_align_score'] = df.apply(lambda x: get_alignment_score(x['Sentence1'],x['Sentence2'],x['Alignment'], 'cd'), axis=1)


/home/aarthy/anaconda3/envs/NLP/lib/python2.7/site-packages/ipykernel_launcher.py:63: RuntimeWarning: invalid value encountered in divide


In [160]:
from sklearn.metrics.pairwise import manhattan_distances,euclidean_distances
from scipy.stats import pearsonr,kendalltau

def get_mahattan(s1,s2):
    s1_emb = getSentenceEmbedings(s1)
    s2_emb = getSentenceEmbedings(s2)
    
    m = manhattan_distances(s1_emb,s2_emb)
    
    return m[0][0]

def get_euclidian(s1,s2):
    s1_emb = getSentenceEmbedings(s1)
    s2_emb = getSentenceEmbedings(s2)
    
    m = euclidean_distances(s1_emb,s2_emb)
    
    return m[0][0]

def get_pearson_correlation(s1,s2):
    s1_emb = getSentenceEmbedings(s1)
    s2_emb = getSentenceEmbedings(s2)
    try:
        m = pearsonr(s1_emb,s2_emb)
        return m[0]
    except:
        pass
    
    return -1

def get_kend_correlation(s1,s2):
    s1_emb = getSentenceEmbedings(s1)
    s2_emb = getSentenceEmbedings(s2)
    
    m = kendalltau(s1_emb,s2_emb)
    
    return m[0]

In [161]:
df['man_dis'] = df.apply(lambda x: get_mahattan(x['Sentence1'], x['Sentence2']), axis=1)
df['euc_dis'] = df.apply(lambda x: get_euclidian(x['Sentence1'], x['Sentence2']), axis=1)
df['pear_dis'] = df.apply(lambda x: get_pearson_correlation(x['Sentence1'], x['Sentence2']), axis=1)
df['kend_dis'] = df.apply(lambda x: get_kend_correlation(x['Sentence1'], x['Sentence2']), axis=1)

In [162]:
df['Bleu_score'] = df.apply(lambda x:  nltk.translate.bleu_score.sentence_bleu(x['Sentence1'],x['Sentence2']), axis=1)

In [163]:
df['len_S1'] = df['Sentence1'].map(len)
df['len_S2'] = df['Sentence2'].map(len)
    
df['S1_diff_S2'] = (df['len_S1'] - df['len_S2'])
df['S2_diff_S1'] = (df['len_S2'] - df['len_S1'])

df['S1_or_S2'] =  df.apply(lambda x: x['len_S1'] | x['len_S2'], axis=1)
df['S1_and_S2'] = df.apply(lambda x: x['len_S1'] & x['len_S2'], axis=1)

df['S1_diff_S2_div_len_S2'] = df['S1_diff_S2'] / df['len_S2'].map(float)
df['S2_diff_A_div_len_S1'] = df['S2_diff_S1'] / df['len_S1'].map(float)


In [164]:
from nltk import pos_tag

def separateTags(tags):
    pos = {'N': set(), 'V': set(), 'J': set(), 'R': set()}
    for t in tags:
        #Just get what the tag starts with 
        (word, tag) = t
        if(tag[0] in ['N', 'V', 'J', 'R']):
            pos[tag[0]].add(word)
    return pos

pos_S1 = df['Sentence1'].map(pos_tag)
pos_S2 = df['Sentence2'].map(pos_tag)

posDF = pd.DataFrame() 
posDF['pos_A'] = pos_S1.map(separateTags)
posDF['pos_B'] = pos_S2.map(separateTags)

df['N_A_diff_B'] =  posDF.apply(lambda x: x['pos_A']['N'] - x['pos_B']['N'], axis=1).map(len)
df['V_A_diff_B'] =  posDF.apply(lambda x: x['pos_A']['V'] - x['pos_B']['V'], axis=1).map(len)
df['J_A_diff_B'] =  posDF.apply(lambda x: x['pos_A']['J'] - x['pos_B']['J'], axis=1).map(len)
df['R_A_diff_B'] =  posDF.apply(lambda x: x['pos_A']['R'] - x['pos_B']['R'], axis=1).map(len)

df['N_B_diff_A'] =  posDF.apply(lambda x: x['pos_B']['N'] - x['pos_A']['N'], axis=1).map(len)
df['V_B_diff_A'] =  posDF.apply(lambda x: x['pos_B']['V'] - x['pos_A']['V'], axis=1).map(len)
df['J_B_diff_A'] =  posDF.apply(lambda x: x['pos_B']['J'] - x['pos_A']['J'], axis=1).map(len)
df['R_B_diff_A'] =  posDF.apply(lambda x: x['pos_B']['R'] - x['pos_A']['R'], axis=1).map(len)

del posDF

In [148]:
# from nltk.corpus import wordnet as wn

# def aAntonymsInb(aSynsets, b):
#     aAntInb = set()                 
#     for syn in aSynsets:
#         for l in syn.lemmas():
#             if l.antonyms() and (l.antonyms()[0].name() in b):
#                 aAntInb.add(l.antonyms()[0].name())
#     return aAntInb

# def penn_to_wn(tag):
#     if tag.startswith('J'):
#         return wn.ADJ
#     elif tag.startswith('N'):
#         return wn.NOUN
#     elif tag.startswith('R'):
#         return wn.ADV
#     elif tag.startswith('V'):
#         return wn.VERB
#     return None

# def getSynset(tag):
#     synset = set()
#     for t in tag:
#         word, tag = t
#         wn_tag = penn_to_wn(tag)
#         if wn_tag is None:
#             continue
#         try:
#             synset.add(wn.synsets(word, wn_tag)[0])
#         except:
#             continue
#     return synset


# negationWords = {'never', 'no', 'nothing', 'nowhere', 'none', 'not', 'n\'t'}

# df['sentence_A'] = df['Sentence1'].map(set)
# df['sentence_B'] = df['Sentence2'].map(set)
    

# #Negative words in each sentence
# df['negDiff'] = df.apply(lambda x: 1 if (len(x['sentence_A'] & negationWords) != len(x['sentence_B'] & negationWords)) else -1, axis=1)
# #pairs['neg_B'] = pairs.apply(lambda x: x['sentence_B'] & negationWords, axis=1).map(len)

# df['synset_A'] = pos_S1.map(getSynset)
# df['synset_B'] = pos_S2.map(getSynset)

# df['A_ant_in_B'] = df.apply(lambda x: aAntonymsInb(x['synset_A'], x['sentence_B']), axis=1).map(len)
# df['B_ant_in_A'] = df.apply(lambda x: aAntonymsInb(x['synset_B'], x['sentence_A']), axis=1).map(len)

# #pairs['negDiff'] = 1 if np.any([neg_A != neg_B]) else -1
# df['contradiction'] = df.apply(lambda x: 1 if (x['negDiff'] or x['A_ant_in_B'] > 0 or x['B_ant_in_A'] > 0) else -1 , axis=1)

In [165]:
df.to_pickle("train_data_features")

In [150]:
df

,Sentence1,Sentence2,Score,Alignment,1gram_woverlap,2gram_woverlap,3gram_woverlap,2gram_coverlap,3gram_coverlap,4gram_coverlap,...,S1_diff_S2_div_len_S2,S2_diff_A_div_len_S1,N_A_diff_B,V_A_diff_B,J_A_diff_B,R_A_diff_B,N_B_diff_A,V_B_diff_A,J_B_diff_A,R_B_diff_A
0,"[boy, play, outdoors, man, smile]","[group, kid, play, yard, old, man, stand, back...",3.300,"[[2, 3], [4, 5], [1, 2]]",11,-1,-1,-1,-1,-1,...,-0.375000,0.600000,4,0,0,0,3,1,1,2
1,"[group, boys, yard, play, man, stand, background]","[young, boys, play, outdoors, man, smile, nearby]",3.700,"[[4, 3], [2, 2], [5, 5]]",7,-1,-1,-1,-1,-1,...,0.000000,0.000000,3,1,0,1,3,0,2,0
2,"[group, child, play, house, man, stand, backgr...","[young, boy, play, outdoors, man, smile, nearby]",3.000,"[[3, 3], [5, 5], [2, 2]]",11,-1,-1,-1,-1,-1,...,0.000000,0.000000,3,2,0,0,3,0,2,0
3,"[brown, dog, attack, another, animal, front, t...","[brown, dog, attack, another, animal, front, m...",4.900,"[[9, 8], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5...",3,3,3,3,3,5,...,0.125000,-0.111111,0,0,1,0,0,0,0,0
4,"[brown, dog, attack, another, animal, front, m...","[brown, dog, help, another, animal, front, man...",3.665,"[[8, 8], [4, 4], [5, 5], [6, 6], [7, 7], [1, 1...",3,3,7,3,7,7,...,0.000000,0.000000,2,0,0,0,1,0,1,0
5,"[two, dog, wrestle, hugging]","[dog, wrestle, hugging]",3.300,"[[4, 3], [2, 1], [3, 2]]",3,3,5,3,5,-1,...,0.333333,-0.250000,0,1,1,0,2,0,0,0
6,"[brown, dog, attack, another, animal, front, m...","[dog, wrestling, hugging]",2.700,"[[2, 1]]",21,-1,-1,-1,-1,-1,...,1.666667,-0.625000,5,0,1,0,2,0,0,0
7,"[two, dog, wrestle, hugging]","[brown, dog, attack, another, animal, front, t...",2.900,"[[2, 2]]",25,-1,-1,-1,-1,-1,...,-0.555556,1.250000,1,1,1,0,6,0,2,0
8,"[two, dog, wrestle, hugging]","[brown, dog, help, another, animal, front, man...",2.300,"[[2, 2]]",23,-1,-1,-1,-1,-1,...,-0.500000,1.000000,1,1,1,0,5,0,2,0
9,"[person, black, jacket, trick, motorbike]","[man, black, jacket, trick, motorbike]",4.900,"[[5, 5], [2, 2], [3, 3], [4, 4], [1, 1]]",3,3,3,3,3,7,...,0.000000,0.000000,1,0,0,0,1,0,0,0
